In [ ]:
# Basic imports
import pandas as pd
import numpy as np


In [ ]:
# We will drop this row
kc_data.drop(15856, axis=0, inplace=True)

In [ ]:
def bath_bed_ratio_outlier(df):
    df.copy()
    df["bath_bed_ratio"] = df["bathrooms"] / df["bedrooms"]
    for idx, ratio in enumerate(df["bath_bed_ratio"]):
        if ratio >= 2:
            df.drop(idx, inplace=True)
        elif ratio <= 0.10:
            df.drop(idx, inplace=True)
    return df

In [ ]:
# We replace "?" with Nan
kc_data['sqft_basement'] = kc_data['sqft_basement'].replace('?', np.NaN)
# And we change the dtype of the column "sqft_basement" to float
kc_data['sqft_basement'] = kc_data['sqft_basement'].astype(float)

In [ ]:
# We are calculating the "sqft_basement" by substracting sqft_above of sqft_living
kc_data.eval('sqft_basement = sqft_living - sqft_above', inplace=True)

**Missings Values**



In [ ]:
# Summation of the missing values and calculation of the missing values as a percentage
missing_values = pd.DataFrame(kc_data.isnull().sum(),columns=['count'])
missing_values['percentage'] = (missing_values['count']/kc_data.shape[0]*100).round(2)
missing_values.query('count != 0')

In [ ]:
# We display how often the different values of the variable occur.
kc_data['view'].value_counts()

In [ ]:
# We replace Nan values in "view" with the most frequent expression (0)
kc_data['view'].fillna(0, inplace=True)

In [ ]:
# We display how often the different values of the variable occur.
kc_data.waterfront.value_counts()

In [ ]:
# We replace Nan values in "waterfront" with the most frequent expression (0)
kc_data.waterfront.fillna(0, inplace=True)

In [ ]:
# We again look at the missing data
missing_values = pd.DataFrame(kc_data.isnull().sum(),columns=['count'])
missing_values['percentage'] = missing_values['count']/kc_data.shape[0]*100
missing_values.query('count != 0')

In [ ]:
# We will create an empty list in which we will store values
last_known_change = []

for idx, yr_re in kc_data.yr_renovated.items():
    
    if str(yr_re) == 'nan' or yr_re == 0.0:
        last_known_change.append(kc_data.yr_built[idx])
    
    else:
        last_known_change.append(int(yr_re))

In [ ]:
# We create a new column and take over the values of our previously created list
kc_data['last_known_change'] = last_known_change

In [ ]:
# We delete the "yr_renovated" and "yr_built" columns
kc_data.drop("yr_renovated", axis=1, inplace=True)
kc_data.drop("yr_built", axis=1, inplace=True)

In [ ]:
# Converting date to 
kc_data['date'] = pd.to_datetime(kc_data['date'])

## Feature Engineering

In [ ]:
# We create a new variable that gives us the price per square foot of living space
kc_data['sqft_price'] = (kc_data.price/(kc_data.sqft_living + kc_data.sqft_lot)).round(2)

In [ ]:
kc_data[kc_data['sqft_price']==kc_data['sqft_price'].max()]

In [ ]:
kc_data[kc_data['price']==kc_data['price'].max()]

In [ ]:
# Absolute difference of latitude between centre and property
kc_data['delta_lat'] = np.absolute(47.62774- kc_data['lat'])
# Absolute difference of longitude between centre and property
kc_data['delta_long'] = np.absolute(-122.24194-kc_data['long'])
# Distance between centre and property
kc_data['center_distance']= ((kc_data['delta_long']* np.cos(np.radians(47.6219)))**2 
                                   + kc_data['delta_lat']**2)**(1/2)*2*np.pi*6378/360

In [ ]:
# This function helps us to calculate the distance between the house overlooking the seafront and the other houses.
def dist(long, lat, ref_long, ref_lat):
    '''dist computes the distance in km to a reference location. Input: long and lat of 
    the location of interest and ref_long and ref_lat as the long and lat of the reference location'''
    delta_long = long - ref_long
    delta_lat = lat - ref_lat
    delta_long_corr = delta_long * np.cos(np.radians(ref_lat))
    return ((delta_long_corr)**2 +(delta_lat)**2)**(1/2)*2*np.pi*6378/360

In [ ]:
# All houses with "waterfront" are added to the list
water_list= kc_data.query('waterfront == 1')
water_list.head()

In [ ]:
water_distance = []
# For each row in our data frame we now calculate the distance to the seafront
for idx, lat in kc_data.lat.items():
    ref_list = []
    for x,y in zip(list(water_list.long), list(water_list.lat)):
        ref_list.append(dist(kc_data.long[idx], kc_data.lat[idx],x,y).min())
    water_distance.append(min(ref_list))

In [ ]:
# wir erstellen eine neue Spalte und übernehmen die Werte unserer vorher erstellten Liste
kc_data['water_distance'] = water_distance